In [25]:
import os
import pandas as pd
from datetime import datetime
import json
import gc


#47729 number of records for kharar with status ACTIVE

# folder_path = '/home/prerna/Punjab/punjab-data-prod-analysis/kharar/ouput_demand_detail/'

# read active properties & needed columns
property_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/chamkaursahib/eg_pt_property.csv',
    usecols=['id', 'propertyid', 'tenantid', 'createdtime', 'additionaldetails', 'ownershipcategory', 'status', 'usagecategory']
)
property_df = property_df[property_df['status'] == 'ACTIVE'].copy()
# property_df = property_df[property_df['propertyid'] == 'PT-1503-2007991'].copy()

# read units
unit_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/chamkaursahib/eg_pt_unit.csv',
    usecols=['propertyid', 'occupancytype']
)

# read demand
demand_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/chamkaursahib/egbs_demand_v1.csv',
    usecols=['id', 'taxperiodfrom', 'taxperiodto', 'consumercode', 'status']
)
demand_df = demand_df[demand_df['status'] == 'ACTIVE'].copy()

# read demand details (memory‑efficient, in chunks)
# all_chunks = []
# needed_cols = ['demandid', 'taxamount', 'collectionamount', 'taxheadcode']
# for filename in os.listdir(folder_path):
#     if filename.endswith('.csv'):
#         file_path = os.path.join(folder_path, filename)
#         print(f'Loading: {file_path}')
#         chunk = pd.read_csv(file_path, usecols=needed_cols)
#         all_chunks.append(chunk)
# demand_details_df = pd.concat(all_chunks, ignore_index=True)

demand_details_df = pd.read_csv('/home/prerna/Punjab/punjab-data-prod-analysis/chamkaursahib/outputDemandDetails.csv', 
                                usecols=['demandid', 'taxamount', 'collectionamount', 'taxheadcode']
                               )
# del all_chunks; gc.collect()

print("✅ Loaded data")

✅ Loaded data


In [26]:
# join pt and unit
joined_pt_unit = property_df.merge(unit_df, left_on='id', right_on='propertyid', how='left', suffixes=('_property', '_unit'))
del property_df, unit_df; gc.collect()
joined_pt_unit.head()
print(demand_df['id'].nunique())

14613


In [27]:
# join demand and demand details
joined_demand = demand_df.merge(demand_details_df, left_on='id', right_on='demandid', how='left', suffixes=('_demand', '_detail'))
print(joined_demand['id'].nunique())
del demand_details_df, demand_df; gc.collect()
joined_demand.head()

14613


,id,consumercode,taxperiodfrom,taxperiodto,status,demandid,taxheadcode,taxamount,collectionamount
0,25318,PT-2002-014095,1522540800000,1554076799000,ACTIVE,25318,PT_OWNER_EXEMPTION,0.0,0.0
1,25318,PT-2002-014095,1522540800000,1554076799000,ACTIVE,25318,PT_TIME_PENALTY,0.0,0.0
2,25318,PT-2002-014095,1522540800000,1554076799000,ACTIVE,25318,PT_CANCER_CESS,18.0,18.0
3,25318,PT-2002-014095,1522540800000,1554076799000,ACTIVE,25318,PT_FIRE_CESS,45.0,45.0
4,25318,PT-2002-014095,1522540800000,1554076799000,ACTIVE,25318,PT_TAX,900.0,900.0


In [28]:
# import pytz
# print(joined_demand.head(10))
# print(joined_demand['taxperiodfrom'].head(10))

# # If not already datetime, convert
# joined_demand['taxperiodfrom'] = pd.to_datetime(joined_demand['taxperiodfrom'], unit='ms', utc=True)
# joined_demand['taxperiodto'] = pd.to_datetime(joined_demand['taxperiodto'], unit='ms', utc=True)

# print(joined_demand['taxperiodfrom'].head(10))

# # Convert to IST (Asia/Kolkata)
# ist = pytz.timezone('Asia/Kolkata')
# joined_demand['taxperiodfrom'] = joined_demand['taxperiodfrom'].dt.tz_convert(ist)
# joined_demand['taxperiodto'] = joined_demand['taxperiodto'].dt.tz_convert(ist)


# def get_fy(date):
#     if date.month >= 4:
#         fy_start = date.year
#         fy_end = date.year + 1
#     else:
#         fy_start = date.year - 1
#         fy_end = date.year
#     return f"{fy_start}-{str(fy_end)[-2:]}"
    
# joined_demand['fy'] = joined_demand['taxperiodfrom'].apply(get_fy)

# result = joined_demand.groupby('consumercode')['fy'].agg(['min', 'max']).reset_index()
# result.rename(columns={'min': 'earliest_fy', 'max': 'latest_fy'}, inplace=True)

# print(result)
import pytz

# Correct: parse as datetime from milliseconds since epoch
joined_demand['taxperiodfrom'] = pd.to_datetime(joined_demand['taxperiodfrom'], unit='ms', utc=True)
joined_demand['taxperiodto'] = pd.to_datetime(joined_demand['taxperiodto'], unit='ms', utc=True)

# Convert to IST (Asia/Kolkata)
ist = pytz.timezone('Asia/Kolkata')
joined_demand['taxperiodfrom'] = joined_demand['taxperiodfrom'].dt.tz_convert(ist)
joined_demand['taxperiodto'] = joined_demand['taxperiodto'].dt.tz_convert(ist)

# Financial year calculation
def get_fy(date):
    if date.month >= 4:
        fy_start = date.year
        fy_end = date.year + 1
    else:
        fy_start = date.year - 1
        fy_end = date.year
    return f"{fy_start}-{str(fy_end)[-2:]}"

joined_demand['fy'] = joined_demand['taxperiodfrom'].apply(get_fy)

# Group by consumercode
result = joined_demand.groupby('consumercode')['fy'].agg(['min', 'max']).reset_index()
result.rename(columns={'min': 'earliest_fy', 'max': 'latest_fy'}, inplace=True)

print(result)


        consumercode earliest_fy latest_fy
0     PT-2002-011056     2018-19   2023-24
1     PT-2002-014069     2018-19   2023-24
2     PT-2002-014095     2018-19   2024-25
3     PT-2002-020623     2014-15   2023-24
4     PT-2002-020630     2014-15   2023-24
...              ...         ...       ...
3398  PT-2002-988755     2014-15   2023-24
3399  PT-2002-992047     2014-15   2023-24
3400  PT-2002-995680     2019-20   2023-24
3401  PT-2002-998548     2014-15   2023-24
3402  PT-2002-998574     2014-15   2023-24

[3403 rows x 3 columns]


In [29]:
# Merge latest_fy onto joined_demand by consumercode
joined = joined_demand.merge(
    result[['consumercode', 'latest_fy']],
    on='consumercode',
    how='left'
)

latest_demand = joined[joined['fy'] == joined['latest_fy']]

demand_sum = latest_demand.groupby('consumercode')['taxamount'].sum().reset_index()
demand_sum.rename(columns={'taxamount':'latest_fy_taxamount'}, inplace=True)

result = result.merge(demand_sum, on='consumercode', how='left')

print(result)

        consumercode earliest_fy latest_fy  latest_fy_taxamount
0     PT-2002-011056     2018-19   2023-24                 96.0
1     PT-2002-014069     2018-19   2023-24                154.0
2     PT-2002-014095     2018-19   2024-25               1053.0
3     PT-2002-020623     2014-15   2023-24               1993.0
4     PT-2002-020630     2014-15   2023-24                887.0
...              ...         ...       ...                  ...
3398  PT-2002-988755     2014-15   2023-24                110.0
3399  PT-2002-992047     2014-15   2023-24                 58.0
3400  PT-2002-995680     2019-20   2023-24                873.0
3401  PT-2002-998548     2014-15   2023-24                140.0
3402  PT-2002-998574     2014-15   2023-24                298.0

[3403 rows x 4 columns]


In [30]:
#Calculating the tax amount(demand) of current year
target_fy = "2025-26"
current_fy_demand = joined_demand[joined_demand['fy'] == target_fy]

df_fy_sum = current_fy_demand.groupby('consumercode')['taxamount'].sum().reset_index()
df_fy_sum.rename(columns={'taxamount': 'current_fy_taxamount'}, inplace=True)

all_consumercodes = pd.DataFrame(joined_demand['consumercode'].unique(), columns=['consumercode'])

final = all_consumercodes.merge(df_fy_sum, on='consumercode', how='left')
final['current_fy_taxamount'] = final['current_fy_taxamount'].fillna(0)

result = result.merge(final, on='consumercode', how='left')
result['current_fy_taxamount'] = result['current_fy_taxamount'].fillna(0)

print(result)

        consumercode earliest_fy latest_fy  latest_fy_taxamount  \
0     PT-2002-011056     2018-19   2023-24                 96.0   
1     PT-2002-014069     2018-19   2023-24                154.0   
2     PT-2002-014095     2018-19   2024-25               1053.0   
3     PT-2002-020623     2014-15   2023-24               1993.0   
4     PT-2002-020630     2014-15   2023-24                887.0   
...              ...         ...       ...                  ...   
3398  PT-2002-988755     2014-15   2023-24                110.0   
3399  PT-2002-992047     2014-15   2023-24                 58.0   
3400  PT-2002-995680     2019-20   2023-24                873.0   
3401  PT-2002-998548     2014-15   2023-24                140.0   
3402  PT-2002-998574     2014-15   2023-24                298.0   

      current_fy_taxamount  
0                      0.0  
1                      0.0  
2                      0.0  
3                      0.0  
4                      0.0  
...                  

In [31]:
# Fiscal years before the current FY
arrear_demand = joined_demand[joined_demand['fy'] < "2025-26"]

agg = arrear_demand.groupby('consumercode').agg(
    arrear_taxamount_sum=('taxamount', 'sum'),
    arrear_collectionamount_sum=('collectionamount', 'sum')
).reset_index()

agg['arrear_years_demand_generated'] = (
    agg['arrear_taxamount_sum'] - agg['arrear_collectionamount_sum']
)

result = result.merge(
    agg[['consumercode', 'arrear_years_demand_generated']],
    on='consumercode', how='left'
)
result['arrear_years_demand_generated'] = result['arrear_years_demand_generated'].fillna(0)

print(result)

        consumercode earliest_fy latest_fy  latest_fy_taxamount  \
0     PT-2002-011056     2018-19   2023-24                 96.0   
1     PT-2002-014069     2018-19   2023-24                154.0   
2     PT-2002-014095     2018-19   2024-25               1053.0   
3     PT-2002-020623     2014-15   2023-24               1993.0   
4     PT-2002-020630     2014-15   2023-24                887.0   
...              ...         ...       ...                  ...   
3398  PT-2002-988755     2014-15   2023-24                110.0   
3399  PT-2002-992047     2014-15   2023-24                 58.0   
3400  PT-2002-995680     2019-20   2023-24                873.0   
3401  PT-2002-998548     2014-15   2023-24                140.0   
3402  PT-2002-998574     2014-15   2023-24                298.0   

      current_fy_taxamount  arrear_years_demand_generated  
0                      0.0                         375.00  
1                      0.0                           0.00  
2              

In [32]:
relevant_codes = ['PT_TIME_PENALTY', 'PT_TIME_INTEREST']
filtered = joined_demand[joined_demand['taxheadcode'].isin(relevant_codes)]

grouped = (
    filtered.groupby(['consumercode', 'taxheadcode'])['taxamount']
    .sum()
    .unstack(fill_value=0)  # Puts taxheadcodes as columns, fills missing with 0
    .reset_index()
)

grouped = grouped[['consumercode', 'PT_TIME_PENALTY', 'PT_TIME_INTEREST']]
grouped = grouped.fillna(0)

result = result.merge(grouped, on='consumercode', how='left')
result[['PT_TIME_PENALTY', 'PT_TIME_INTEREST']] = result[['PT_TIME_PENALTY', 'PT_TIME_INTEREST']].fillna(0)

print(result)

        consumercode earliest_fy latest_fy  latest_fy_taxamount  \
0     PT-2002-011056     2018-19   2023-24                 96.0   
1     PT-2002-014069     2018-19   2023-24                154.0   
2     PT-2002-014095     2018-19   2024-25               1053.0   
3     PT-2002-020623     2014-15   2023-24               1993.0   
4     PT-2002-020630     2014-15   2023-24                887.0   
...              ...         ...       ...                  ...   
3398  PT-2002-988755     2014-15   2023-24                110.0   
3399  PT-2002-992047     2014-15   2023-24                 58.0   
3400  PT-2002-995680     2019-20   2023-24                873.0   
3401  PT-2002-998548     2014-15   2023-24                140.0   
3402  PT-2002-998574     2014-15   2023-24                298.0   

      current_fy_taxamount  arrear_years_demand_generated  PT_TIME_PENALTY  \
0                      0.0                         375.00             0.00   
1                      0.0             

In [33]:
unit_all_columns_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/chamkaursahib/eg_pt_unit.csv'
)
print(unit_all_columns_df)

                                        id          tenantid  \
0     b14e2056-9cb8-4bc2-9bc5-949e7e2769a8  pb.chamkaursahib   
1     64f19d9c-a090-45f1-ad55-9baf8714914a  pb.chamkaursahib   
2     f2917ea2-878c-4ce7-9bbd-e50103e8f882  pb.chamkaursahib   
3     bce1365b-7680-4533-b09f-e25fe9f99210  pb.chamkaursahib   
4     cdc765f6-7722-4e79-8dae-9e33dc83d363  pb.chamkaursahib   
...                                    ...               ...   
7149  b44b5ab7-2df4-4c24-a4a3-9df1ab0fd37a  pb.chamkaursahib   
7150  7fc3dbac-b279-4532-aab6-3bcbb9beefc6  pb.chamkaursahib   
7151  6b714e1e-ba3f-4e07-8b18-27a70587bed1  pb.chamkaursahib   
7152  4d0bfcd1-c053-458e-a129-0b7a9f5bbe0e  pb.chamkaursahib   
7153  d3f587cc-471d-4f3b-890a-2de3225b3e93  pb.chamkaursahib   

                                propertyid  floorno         unittype  \
0     196d8ee8-2fce-4fd1-8f05-c6a8b203a7cc        0  OTHERCOMMERCIAL   
1     1e7c7d31-6047-4bc8-a85c-c13751a550e8        0  OTHERCOMMERCIAL   
2     1e7c7d31-

In [34]:

# read active properties & needed columns
property_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/chamkaursahib/eg_pt_property.csv',
    usecols=['id', 'propertyid', 'tenantid', 'createdtime', 'additionaldetails', 'ownershipcategory', 'status', 'usagecategory', 'propertytype']
)
property_df = property_df[property_df['status'] == 'ACTIVE'].copy()
# Merge properties and units by property id
merged = property_df.merge(unit_all_columns_df, left_on='id', right_on='propertyid', suffixes=('_property', '_unit'))

def classify_ownership(occupancies):
    unique_types = set(occupancies)
    if 'RENTED' in unique_types:
        if len(unique_types) > 1:
            return 'Mixed'
        else:
            return 'Tenant'
    if 'SELFOCCUPIED' in unique_types:
        # If only SELFOCCUPIED or SELFOCCUPIED + UNOCCUPIED
        return 'Owner'
    if 'UNOCCUPIED' in unique_types:
        return 'Owner'
    # fallback
    return None

# Find occupancytypes per property id
ownership = (
    merged.groupby('propertyid_property')['occupancytype']
    .apply(classify_ownership)
    .reset_index()
    .rename(columns={'occupancytype': 'Owned_Rented'})
)

property_df = property_df.merge(ownership, left_on='propertyid', right_on = 'propertyid_property', how='left')

print(property_df)


                                        id       propertyid          tenantid  \
0     24affb1d-5740-49bd-8ebc-844ee573ea2b  PT-2002-2008857  pb.chamkaursahib   
1     cf46cb05-c777-4ee8-96aa-b75b429fb715  PT-2002-2032655  pb.chamkaursahib   
2     e8626877-333d-4b6a-bee1-d975ca6a207c  PT-2002-2109515  pb.chamkaursahib   
3     f0c3431a-5fdb-43aa-8940-004584f8673b  PT-2002-1031373  pb.chamkaursahib   
4     19fd36fb-2efd-4407-ab67-a706906259cb  PT-2002-1033429  pb.chamkaursahib   
...                                    ...              ...               ...   
3351  1296c4d2-3458-4b07-a5dc-ec8286ebfb72  PT-2002-1625859  pb.chamkaursahib   
3352  7336502e-7e09-401c-b16e-bb6f569cc9af  PT-2002-1696248  pb.chamkaursahib   
3353  e6dd6cfd-b013-4d99-9f92-2c6c88e620f0  PT-2002-1625870  pb.chamkaursahib   
3354  c6f7fc05-a7c2-4736-8c27-1c4dc6fa2abc  PT-2002-1986982  pb.chamkaursahib   
3355  97c28894-e91c-4027-8660-41d2039c4658  PT-2002-1993793  pb.chamkaursahib   

      status               

In [35]:
# # def clean_numeric(series):
# #     # Replace 'NULL' strings and NaNs with 0, then convert to float
# #     return pd.to_numeric(series.replace('NULL', 0), errors='coerce').fillna(0)

# # unit_all_columns_df['builtuparea'] = clean_numeric(unit_all_columns_df['builtuparea'])
# # unit_all_columns_df['plintharea'] = clean_numeric(unit_all_columns_df['plintharea'])

# # area_summary = (
# #     unit_all_columns_df.groupby('propertyid')
# #     .agg(
# #         total_builtup_area=('builtuparea', 'sum'),
# #         total_plinth_area=('plintharea', 'sum')
# #     )
# #     .reset_index()
# # )

# # property_df = property_df.merge(area_summary, left_on='id', right_on='propertyid', how='left')
# # print(property_df)
# # property_df['total_builtup_area'] = property_df['total_builtup_area'].fillna(0)
# # property_df['total_plinth_area'] = property_df['total_plinth_area'].fillna(0)
# total_plinth_area

def clean_numeric(series):
    # Replace 'NULL' strings and NaNs with 0, then convert to float
    return pd.to_numeric(series.replace('NULL', 0), errors='coerce').fillna(0)

merged['builtuparea'] = clean_numeric(merged['builtuparea'])
merged['plintharea'] = clean_numeric(merged['plintharea'])

area_summary = (
    merged.groupby('propertyid_property', as_index=False)
    .agg(
        total_builtup_area=('builtuparea', 'sum'),
        total_plinth_area=('plintharea', 'sum')
    )
)
# for col in ['total_builtup_area', 'total_plinth_area']:
#     if col in property_df.columns:
#         property_df = property_df.drop(col, axis=1)

property_df = property_df.merge(area_summary, left_on='propertyid' ,right_on='propertyid_property', how='left')
property_df['total_builtup_area'] = property_df['total_builtup_area'].fillna(0)
property_df['total_plinth_area'] = property_df['total_plinth_area'].fillna(0)

print(property_df)

                                        id       propertyid          tenantid  \
0     24affb1d-5740-49bd-8ebc-844ee573ea2b  PT-2002-2008857  pb.chamkaursahib   
1     cf46cb05-c777-4ee8-96aa-b75b429fb715  PT-2002-2032655  pb.chamkaursahib   
2     e8626877-333d-4b6a-bee1-d975ca6a207c  PT-2002-2109515  pb.chamkaursahib   
3     f0c3431a-5fdb-43aa-8940-004584f8673b  PT-2002-1031373  pb.chamkaursahib   
4     19fd36fb-2efd-4407-ab67-a706906259cb  PT-2002-1033429  pb.chamkaursahib   
...                                    ...              ...               ...   
3351  1296c4d2-3458-4b07-a5dc-ec8286ebfb72  PT-2002-1625859  pb.chamkaursahib   
3352  7336502e-7e09-401c-b16e-bb6f569cc9af  PT-2002-1696248  pb.chamkaursahib   
3353  e6dd6cfd-b013-4d99-9f92-2c6c88e620f0  PT-2002-1625870  pb.chamkaursahib   
3354  c6f7fc05-a7c2-4736-8c27-1c4dc6fa2abc  PT-2002-1986982  pb.chamkaursahib   
3355  97c28894-e91c-4027-8660-41d2039c4658  PT-2002-1993793  pb.chamkaursahib   

      status               

In [36]:
# for col in ['total_builtup_area_x', 'total_plinth_area_x', 'total_builtup_area_y', 'total_plinth_area_y', 'propertyid_property_y', 'propertyid_property', 'total_builtup_area', 'total_plinth_area']:
#     if col in property_df.columns:
#         property_df = property_df.drop(col, axis=1)

# print(property_df)

In [37]:
property_result_merged = property_df.merge(
    result,
    left_on='propertyid',
    right_on='consumercode',
    how='left'
)

print(property_result_merged)

                                        id       propertyid          tenantid  \
0     24affb1d-5740-49bd-8ebc-844ee573ea2b  PT-2002-2008857  pb.chamkaursahib   
1     cf46cb05-c777-4ee8-96aa-b75b429fb715  PT-2002-2032655  pb.chamkaursahib   
2     e8626877-333d-4b6a-bee1-d975ca6a207c  PT-2002-2109515  pb.chamkaursahib   
3     f0c3431a-5fdb-43aa-8940-004584f8673b  PT-2002-1031373  pb.chamkaursahib   
4     19fd36fb-2efd-4407-ab67-a706906259cb  PT-2002-1033429  pb.chamkaursahib   
...                                    ...              ...               ...   
3351  1296c4d2-3458-4b07-a5dc-ec8286ebfb72  PT-2002-1625859  pb.chamkaursahib   
3352  7336502e-7e09-401c-b16e-bb6f569cc9af  PT-2002-1696248  pb.chamkaursahib   
3353  e6dd6cfd-b013-4d99-9f92-2c6c88e620f0  PT-2002-1625870  pb.chamkaursahib   
3354  c6f7fc05-a7c2-4736-8c27-1c4dc6fa2abc  PT-2002-1986982  pb.chamkaursahib   
3355  97c28894-e91c-4027-8660-41d2039c4658  PT-2002-1993793  pb.chamkaursahib   

      status               

In [23]:
property_result_merged.to_csv('result_with_areas.csv', index= False)

In [38]:
pt_owner_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/chamkaursahib/eg_pt_owner.csv',
    usecols=['propertyid', 'ownertype']
)
print(pt_owner_df.shape)

exempt_types = {'FREEDOMFIGHTER', 'WIDOW'}

# For each propertyid, check if ANY owner has an exempt type
exempt_flags = (
    pt_owner_df.groupby('propertyid')['ownertype']
    .apply(lambda x: any(o in exempt_types for o in x))
    .reset_index()
    .rename(columns={'ownertype': 'Is property exempted'})
)

property_result_merged = property_result_merged.merge(
    exempt_flags,
    left_on='id',
    right_on='propertyid',
    how='left'
)

# Convert True/False to 'Yes'/'No', fill missing (i.e., no owners) as 'No'
property_result_merged['Is property exempted'] = (
    property_result_merged['Is property exempted'].fillna(False)
    .map({True: 'Yes', False: 'No'})
)


(4213, 2)


/tmp/ipykernel_446346/3668072852.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  property_result_merged['Is property exempted'].fillna(False)


In [39]:
property_result_merged.head()

,id,propertyid_x,tenantid,status,propertytype,ownershipcategory,usagecategory,createdtime,additionaldetails,propertyid_property_x,...,consumercode,earliest_fy,latest_fy,latest_fy_taxamount,current_fy_taxamount,arrear_years_demand_generated,PT_TIME_PENALTY,PT_TIME_INTEREST,propertyid_y,Is property exempted
0,24affb1d-5740-49bd-8ebc-844ee573ea2b,PT-2002-2008857,pb.chamkaursahib,ACTIVE,BUILTUP.INDEPENDENTPROPERTY,INDIVIDUAL.SINGLEOWNER,NONRESIDENTIAL.COMMERCIAL,1725258023590,"{""businessName"": ""sunny trading company"", ""yea...",PT-2002-2008857,...,PT-2002-2008857,2023-24,2024-25,2540.0,0.0,0.0,498.72,190.61,24affb1d-5740-49bd-8ebc-844ee573ea2b,No
1,cf46cb05-c777-4ee8-96aa-b75b429fb715,PT-2002-2032655,pb.chamkaursahib,ACTIVE,BUILTUP.INDEPENDENTPROPERTY,INDIVIDUAL.SINGLEOWNER,RESIDENTIAL,1734670498004,"{""yearConstruction"": null}",PT-2002-2032655,...,PT-2002-2032655,2024-25,2024-25,0.0,0.0,0.0,0.00,0.00,cf46cb05-c777-4ee8-96aa-b75b429fb715,Yes
2,e8626877-333d-4b6a-bee1-d975ca6a207c,PT-2002-2109515,pb.chamkaursahib,ACTIVE,BUILTUP.INDEPENDENTPROPERTY,INDIVIDUAL.MULTIPLEOWNERS,MIXED,1748849230399,"{""businessName"": ""MEHTA ELECTRICAL SHOP"", ""yea...",PT-2002-2109515,...,PT-2002-2109515,2023-24,2025-26,1782.0,1782.0,0.0,0.00,0.00,e8626877-333d-4b6a-bee1-d975ca6a207c,No
3,f0c3431a-5fdb-43aa-8940-004584f8673b,PT-2002-1031373,pb.chamkaursahib,ACTIVE,BUILTUP.INDEPENDENTPROPERTY,INDIVIDUAL.SINGLEOWNER,RESIDENTIAL,1614924905047,NaN,PT-2002-1031373,...,PT-2002-1031373,2014-15,2023-24,233.0,0.0,454.0,306.79,809.22,f0c3431a-5fdb-43aa-8940-004584f8673b,No
4,19fd36fb-2efd-4407-ab67-a706906259cb,PT-2002-1033429,pb.chamkaursahib,ACTIVE,BUILTUP.INDEPENDENTPROPERTY,INDIVIDUAL.SINGLEOWNER,RESIDENTIAL,1615788499831,NaN,PT-2002-1033429,...,PT-2002-1033429,2014-15,2023-24,62.0,0.0,184.0,76.05,218.49,19fd36fb-2efd-4407-ab67-a706906259cb,No


In [40]:
# build final report: rename & reorder
report = property_result_merged.rename(columns={
    'tenantid': 'ULB',
    'propertyid_x': 'Property ID',
    'usagecategory': 'Usage',
    'createdtime': 'Date of Creation of the Property in the System',
    'additionaldetails': 'Date of Construction of the Property',
    'ownershipcategory': 'Ownership Type',
    'Owned_Rented': 'Owned_Rented (Owner/ Rented/ Mixed)',
    'earliest_fy': 'Earliest Financial Year for which Demand was Generated',
    'latest_fy': 'Latest Financial Year for which Demand was Generated',
    'latest_fy_taxamount': 'Latest Demand Generated [in Rs.]',
    'current_fy_taxamount': 'Current Years Demand Generated [in Rs.]',
    'PT_TIME_PENALTY': 'Penalty',
    'PT_TIME_INTEREST': 'Interest',
    'arrear_years_demand_generated': 'Arrear Years Demand Generated [in Rs.]',
    'propertytype' : 'Property Type[Building/ Vacant]',
    'total_builtup_area' : 'Total Builtup Area [Sum of all units/ floors]',
    'total_plinth_area' : 'Total Plinth Area [Sum of all units/ floors]',
    'Is property exempted' : 'Is Property Exempted [Yes/ No]'
}).copy()

def epoch_to_custom_date(epoch_ms):
    return datetime.fromtimestamp(epoch_ms / 1000).strftime('%d-%b-%Y') if pd.notna(epoch_ms) else None

def get_year_construction(val):
    if pd.isna(val): return None
    try: return json.loads(val).get('yearConstruction')
    except: return None

# format ULB & dates
report['ULB'] = report['ULB'].str.split('.').str[1].str.capitalize()
report['Date of Creation of the Property in the System'] = report['Date of Creation of the Property in the System'].apply(epoch_to_custom_date)
report['Date of Construction of the Property'] = report['Date of Construction of the Property'].apply(get_year_construction)

# then keep only those columns, in the exact order you want:
final_report = report[
    [
        'ULB',
        'Property ID',
        'Usage',
        'Date of Creation of the Property in the System',
        'Date of Construction of the Property',
        'Ownership Type',
        'Is Property Exempted [Yes/ No]',
        'Owned_Rented (Owner/ Rented/ Mixed)',
        'Earliest Financial Year for which Demand was Generated',
        'Latest Financial Year for which Demand was Generated',
        'Latest Demand Generated [in Rs.]',
        'Current Years Demand Generated [in Rs.]',
        'Penalty',
        'Interest',
        'Arrear Years Demand Generated [in Rs.]',
        'Property Type[Building/ Vacant]',
        'Total Builtup Area [Sum of all units/ floors]',
        'Total Plinth Area [Sum of all units/ floors]'
    ]
].copy()

print("✅ Writing CSV")
final_report.to_csv('Punjab_Data_Analysis_sahib.csv', index=False)
print(f"🎉 Done! CSV generated with {len(final_report)} properties")

✅ Writing CSV
🎉 Done! CSV generated with 3356 properties


In [32]:
# joined_demand[joined_demand['consumercode'] == 'PT-1503-1994032']
joined_demand[joined_demand['consumercode'].str.contains('PT-1503-011887', na=False)]

# joined_demand.head()
print(joined_demand.shape)

print(joined_demand['id'].nunique())

(3490242, 10)
285210


In [33]:
demand_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/kharar/egbs_demand_v1.csv',
    usecols=['id', 'taxperiodfrom', 'taxperiodto', 'consumercode', 'status']
)
demand_df = demand_df[demand_df['status'] == 'ACTIVE'].copy()

print(demand_df['id'].nunique())

287351
